# Extracellular Recording

Extracellular potentials created by given neural networks can be determined at arbitary positions in 3D space. With large neural networks, the potential at a given point is a reasonable approximation to spikes and local field potentials with realistic physiological noise models.

## Simulation 1: Grid of extracellular potentials from a single neuron

In [ ]:
using JNeuron, PyPlot;

#Load 3D Neurolucida file
myimport=input("./data/cell2.asc");

#Generate instance of neuron type with appropriate sections from 3D data
blank_neuron=instantiate(myimport);

#Create segments based on lambda rule
set_nsegs!(blank_neuron);

#add HH and Passive to Soma and Axon, Passive to basal and apical Dendrites
myneuron=add(blank_neuron,(HH(),Passive()),(HH(),Passive()),Passive(),Passive());

#Create network with neurons and simulation stop time of 15.0 ms
mynetwork=Network(myneuron,6.0);

#Add electrodes at grid locations
for x in 0.0:250.0:1000.0
    for y in -1000.0:250.0:1500.0
        
        #Create electrode and add it to the network
        electrode=Extracellular([x,y,0.0]);
        add!(mynetwork,electrode);
        
    end
end

#Create intracellular stimulation at soma and add it to the network
mystim=Stim(5.0,1,1,924,1.0,2.0)
add!(mynetwork,mystim)

In [ ]:
run!(mynetwork);

In [ ]:
#Plot 3D structure of cell
for i=1:length(myimport.sections)
    plot(myimport.sections[i].raw[:,1],myimport.sections[i].raw[:,2],color="black")
end

#Plot extracellular potentials at appropriate locations
for i=1:length(mynetwork.extra)
    ex=mynetwork.extra[i]
    xs=(ex.xyz[1]-round(Int,length(ex.v)/2)):1.0:(ex.xyz[1]+round(Int,length(ex.v)/2))
    ys=ex.xyz[2]
    plot(xs,ex.v.*2000+ys, color="blue",alpha=maximum(abs(ex.v))*3)
end

## Simulation 2: Grid of extracellular potentials from multiple neurons

In [ ]:
using JNeuron, PyPlot, PyCall;
@pyimport matplotlib.collections as C

#Load 3D Neurolucida file
myimport=input("./data/cell2.asc");

#Generate instance of neuron type with appropriate sections from 3D data
blank_neuron=instantiate(myimport);

#Create segments based on lambda rule
set_nsegs!(blank_neuron);

#add HH and Passive to Soma and Axon, Passive to basal and apical Dendrites
myneuron1=add(blank_neuron,(HH(),Passive()),(HH(),Passive()),Passive(),Passive());

myneuron2=deepcopy(myneuron1)
JNeuron.translate3d!(myneuron2,400.0,-1000.0,0.0)
JNeuron.randomize_shape!(myneuron2)

myneuron3=deepcopy(myneuron1)
JNeuron.translate3d!(myneuron3,-400.0,-1000.0,0.0)
JNeuron.randomize_shape!(myneuron3)

#Create network with neurons and simulation stop time of 15.0 ms
mynetwork=Network([myneuron1,myneuron2,myneuron3],6.0);

#Add electrodes at grid locations
for x in 0.0:250.0:1500.0
    for y in -1500.0:250.0:1500.0
        
        #Create electrode and add it to the network
        electrode=Extracellular([x,y,0.0]);
        add!(mynetwork,electrode);
        
    end
end

#Create intracellular stimulation at soma and add it to the network
mystim=Stim(5.0,1,1,924,1.0,2.0)
add!(mynetwork,mystim)

mystim=Stim(5.0,1,2,924,2.0,3.0)
add!(mynetwork,mystim)

mystim=Stim(5.0,1,2,924,3.0,4.0)
add!(mynetwork,mystim)

In [ ]:
run!(mynetwork);

In [ ]:
#Plot 3D structure of cell
(x1,y1)=JNeuron.plot_arrays(myneuron1)
(x2,y2)=JNeuron.plot_arrays(myneuron2)
(x3,y3)=JNeuron.plot_arrays(myneuron3)

for i=1:length(x1)
    plot(x1[i],y1[i],color="black")
end
for i=1:length(x2)
    plot(x2[i],y2[i],color="black")
end
for i=1:length(x3)
    plot(x3[i],y3[i],color="black")
end

#Plot extracellular potentials at appropriate locations
for i=1:length(mynetwork.extra)
    ex=mynetwork.extra[i]
    xs=(ex.xyz[1]-round(Int,length(ex.v)/2)):1.0:(ex.xyz[1]+round(Int,length(ex.v)/2))
    ys=ex.xyz[2]
    plot(xs,ex.v.*2000+ys, color="blue",alpha=maximum(abs(ex.v))*3)
end